# Neural Network Model [[참고사이트]](https://iamtrask.github.io//2015/07/12/basic-python-network/)

## 1. training_data 생성
* Stemming: 단어에서 접사 제거

Stemming algorithm에서 가장 유명한 알고리즘은  
Martin Porter의 **Porter stemming algorithm** 이다.  <br>
이 알고리즘은 영어의 접미사를 제거한다. ex) cooking -> cook

또 다른 알고리즘으로는 Lancaster 대학이 개발한 **Lancaster stemming algorithm**이다.  
LancasterStemmer함수는 PorterStemmer와 비슷한데 성능이 더 좋다.

우리는 여기서 LancasterStemmer함수를 사용했다.

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime

stemmer = LancasterStemmer()

In [2]:
# Test: 3개의 class
training_data = []
training_data.append({"class":"happy", "sentence":"how are you?"})
training_data.append({"class":"happy", "sentence":"how is your day?"})
training_data.append({"class":"happy", "sentence":"good day"})
training_data.append({"class":"happy", "sentence":"how is it going today?"})

training_data.append({"class":"sad", "sentence":"have a nice day"})
training_data.append({"class":"sad", "sentence":"see you later"})
training_data.append({"class":"sad", "sentence":"have a nice day"})
training_data.append({"class":"sad", "sentence":"talk to you soon"})

training_data.append({"class":"fear", "sentence":"make me a sandwich"})
training_data.append({"class":"fear", "sentence":"can you make a sandwich?"})
training_data.append({"class":"fear", "sentence":"having a sandwich today?"})
training_data.append({"class":"fear", "sentence":"what's for lunch?"})


print ("%s sentences in training data" % len(training_data))

12 sentences in training data


## 2. words, classes, documents 만들기
* words: 단어장. 문장을 토큰화하여 해당 결과물을 단어장에 저장
* classes: 클래스. 여기 프로젝트에서는 classes 내용물이 **happy, sad, angry, tired, fear**로 구성
* documents: 위의 training_data에서 문장들을 한군데에 저장
* ignore_words: 무시할 단어

In [3]:
import nltk
nltk.download('punkt')

words = [] # 단어 저장할 리스트
classes = [] #class 저장할 리스트
documents = [] # class + 단어 저장할 리스트
ignore_words = ['?', '!'] # 무시할 단어

for pattern in training_data:
    # 문장 토큰화
    w = nltk.word_tokenize(pattern['sentence'])
    # words리스트에 토큰 추가
    words.extend(w)
    # documents리스트에 class값과 토큰 추가
    documents.append((w, pattern['class']))
    # classes리스트를 검사하여 현재 검사하고 있는 문장의 class가 리스트에 없으면 classes리스트에 추가
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        
# 무시할 단어인지 확인 후 대문자로 이루어진 단어는 소문자로 변경
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
# words리스트에서 중복 제거
words = list(set(words)) 

# classes 리스트에서 중복 제거
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

12 documents
3 classes ['fear', 'happy', 'sad']
26 unique stemmed words ['mak', 'nic', 'what', 'ar', 'lat', 'day', 'to', 'hav', 'see', 'for', 'sandwich', 'is', 'it', 'me', 'how', 'going', 'today', 'a', 'good', "'s", 'can', 'lunch', 'yo', 'soon', 'you', 'talk']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\김경남\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3. 모델 제작
**2개의 layer로 구성된 Neural Network 구성**  
1. numpy: 행렬 곱셈 이용
2. sigmoid함수: 오차 측정을 최소한으로 줄여 예측값을 더욱 정확하게 조정
3. 입력 문장들을 단어별로 0과 1로 변형


### 참고
* **Bow**: Bag-of-words. 글에 어떠한 단어가 있는지 수치로 나타내는 방법

In [5]:
import numpy as np
import time

# sigmoid를 사용하기 위해 함수 선언
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# 결과값에 
def sigmoid_output_to_derivative(output):
    return output*(1-output)

# 문장 토큰화 및 stem화
def clean_up_sentence(sentence):
    # tokenize화
    sentence_words = nltk.word_tokenize(sentence)
    # 단어 stem화
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# bow값 반환: 문장에 해당 단어가 있으면 1 반환, 아니면 0 반환
def bow(sentence, words, show_details=False):
    # clean_up_sentence()를 사용하여 문장 토큰화 및 stem화
    sentence_words = clean_up_sentence(sentence)
    # bow 제작
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # layer0: 입력값들은 words
    l0 = x
    # layer1: layer0과 synapse0 행렬곱
    l1 = sigmoid(np.dot(l0, synapse_0))
    # layer2: layer1과 synapse1 행렬곱 이후 리턴
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [6]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # weights값을 0으로 랜덤초기화
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
        
    for j in iter(range(epochs+1)):

        # layers0, layer1, layer2에 입력값 전달
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # target value 오차 측정
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # layer2: target value값 이 원하는 방향으로 나왔는지 확인, 맞다면 최소한의 변화만 주기
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # weights가 layer1, layer2에 각각 얼마나 기여했는지 계산
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # ilayer1: target value값 이 원하는 방향으로 나왔는지 확인, 맞다면 최소한의 변화만 주기
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # synapse(model) 만들기
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [7]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

delta after 10000 iterations:0.006409989850771434
delta after 20000 iterations:0.004385659225875107
delta after 30000 iterations:0.003521350902361076
delta after 40000 iterations:0.003016270577101034
delta after 50000 iterations:0.002676176721750477
delta after 60000 iterations:0.0024276267370422315
delta after 70000 iterations:0.0022359753059778497
delta after 80000 iterations:0.002082491220730463
delta after 90000 iterations:0.0019560557267743663
delta after 100000 iterations:0.0018496021788299377
saved synapses to: synapses.json
processing time: 7.51242208480835 seconds


In [8]:
ERROR_THRESHOLD = 0.2

synapse_file = 'synapses.json' 

with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

classify("sudo make me a sandwich")
classify("how are you today?")
classify("talk to you tomorrow")
classify("who are you?")
classify("make me some lunch")
classify("how was your lunch today?")
print()
classify("good day", show_details=True)

sudo make me a sandwich 
 classification: [['fear', 0.9985140979666376]]
how are you today? 
 classification: [['happy', 0.9980176847662942]]
talk to you tomorrow 
 classification: [['sad', 0.9860086967337713]]
who are you? 
 classification: [['happy', 0.8260345140800139]]
make me some lunch 
 classification: [['fear', 0.966998884559231]]
how was your lunch today? 
 classification: [['happy', 0.938524906701882], ['fear', 0.2157094633600867]]

found in bag: good
found in bag: day
sentence: good day 
 bow: [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
good day 
 classification: [['happy', 0.9971066514456564]]


[['happy', 0.9971066514456564]]